# Homework 3
Hongye Li, Xiaochuan Ma

## Problem 1

In [1]:
!pip install scikit-image

  Using cached scikit-image-0.19.2.tar.gz (22.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached imageio-2.19.2-py3-none-any.whl (3.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 19.1 MB/s eta 0:00:0000:0100:01
  Using cached networkx-2.8.1-py3-none-any.whl (2.0 MB)
  Using cached tifffile-2022.5.4-py3-none-any.whl (195 kB)
  Created wheel for scikit-image: filename=scikit_image-0.19.2-cp38-cp38-macosx_11_0_arm64.whl size=12173664 sha256=979b9b0eed657be90be4875e36e233213c1d4457e8e68c9fee32c5bf12903f37
  Stored in directory: /Users/lihongye/Library/Caches/pip/wheels/ea/a1/bc/c81156493837d37793f73584eea3d414448636e50033d16627
Successfully built scikit-image
You should consider upgrading via the '/Users/lihongye/miniforge3/envs/mltensor/bin/python -m pip install --upgrade pip' command.


In [38]:
from tensorflow.keras.datasets import mnist
import numpy as np
import skimage.measure
from skimage.util import view_as_blocks
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
import math

In [39]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [40]:
# Rescale the data
train_X=train_X/255
test_X=test_X/255

In [41]:
# indices of data with label 0-4
train_ind=np.where(train_y<5)[0]
test_ind=np.where(test_y<5)[0]
ind_list=[]
for i in range(5):
    ind_list.append(np.where(train_y==i)[0])

In [42]:
#compress the image
train_X_comp=skimage.measure.block_reduce(train_X, (1,2,2), np.mean)
test_X_comp=skimage.measure.block_reduce(test_X, (1,2,2), np.mean)
#image to vector
train_X_comp=np.transpose(train_X_comp, (0, 2, 1))
test_X_comp=np.transpose(test_X_comp, (0, 2, 1))
train_X_vec=train_X_comp.reshape((train_X_comp.shape[0], 196))
test_X_vec=test_X_comp.reshape((test_X_comp.shape[0], 196))
#select the data with label 0-4
train_X_vec_selected=train_X_vec[train_ind]
train_y_selected=train_y[train_ind]
test_X_vec_selected=test_X_vec[test_ind]
test_y_selected=test_y[test_ind]

In [43]:
test_X_vec_selected.shape

(5139, 196)

## Problem 3

### Mixture of spherical Gaussians

In [44]:
##initialization
K=5
N=train_X_vec_selected.shape[0]
d=196
mu=np.random.rand(K,d)
sigma=np.random.rand(K,1)
Pi=np.ones(K)/5
ind_list1=[]
for i in range(5):
    ind_list1.append(np.where(train_y_selected==i)[0])
##EM algorithm
L_old=1
Temp1=np.sum(train_X_vec_selected*train_X_vec_selected,axis=1,keepdims=True)
for iter1 in range(100):
    print(iter1)
    #E step
    M=(-0.5*(1/sigma)@Temp1.transpose()+(mu/sigma)@train_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi*sigma)-0.5*(np.sum(mu*mu/sigma,axis=1,keepdims=True))).transpose()
    Max_M=np.max(M, axis=1,keepdims=True)
    F=M-Max_M
    F=np.exp(F)
    L=np.sum(F,axis=1,keepdims=True)
    F=F/L
    F+=0.00001*np.ones(F.shape)# add a small noise avoid 0 denorminator
    F=F/np.sum(F,axis=1,keepdims=True)
    #Log likelyhood
    L=np.log(L)
    L=np.sum(L)+np.sum(Max_M)
    if abs(L_old-L)/abs(L)<0.0001:
        break
    L_old=L
    #M_step
    Sum_F = np.sum(F,axis=0,keepdims=True)
    Pi=Sum_F/N
    mu=F.transpose()@train_X_vec_selected
    mu=mu/Sum_F.transpose()
    Temp2 =-2*np.sum(F*(train_X_vec_selected@mu.transpose()),axis=0,keepdims=True)
    Temp3=Sum_F*(np.sum(mu*mu,axis=1,keepdims=True)).transpose()
    sigma = F.transpose()@Temp1+Temp2.transpose()+Temp3.transpose()
    sigma = sigma/Sum_F.transpose()/d
##Figure our how the groups of GMM correspond to true lable
train_cluster_result=np.argmax(F,axis=1)
G_to_L=[]
for k1 in range(K):
    temp=train_cluster_result[ind_list1[k1]]
    current=0
    relationship=[k1,k1]
    for k2 in range(K):
        if np.sum(temp==k2)>current:
            current=np.sum(temp==k2)
            relationship[1]=k2
    G_to_L.append(relationship)    
#testing
N_test=test_X_vec_selected.shape[0]
test_Temp1=np.sum(test_X_vec_selected*test_X_vec_selected,axis=1,keepdims=True)
test_M=(-0.5*(1/sigma)@test_Temp1.transpose()+(mu/sigma)@test_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi*sigma)-0.5*(np.sum(mu*mu/sigma,axis=1,keepdims=True))).transpose()
test_Max_M=np.max(test_M, axis=1,keepdims=True)
test_F=test_M-test_Max_M
test_F=np.exp(test_F)
test_L=np.sum(test_F,axis=1,keepdims=True)
test_F=test_F/test_L
test_L=np.log(test_L)
test_L=np.sum(test_L)+np.sum(test_Max_M)
test_result=np.argmax(test_F,axis=1)
trans_test_y_selected=np.zeros([N_test])
for i in range(N_test):
    trans_test_y_selected[i]=G_to_L[test_y_selected[i]][1]
print("The testing error is ", 1-np.sum(test_result==trans_test_y_selected)/N_test)
print("The Log Likelyhood is ",test_L)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
The testing error is  0.11908931698774083
The Log Likelyhood is  247738.3607195337


In [47]:
##500 experiments
error_rate_list1=[]
for _ in range(500):
    ##initialization
    K=5
    N=train_X_vec_selected.shape[0]
    d=196
    mu=np.random.rand(K,d)
    sigma=np.random.rand(K,1)
    Pi=np.ones(K)/5
    ind_list1=[]
    for i in range(5):
        ind_list1.append(np.where(train_y_selected==i)[0])
    ##EM algorithm
    L_old=1
    Temp1=np.sum(train_X_vec_selected*train_X_vec_selected,axis=1,keepdims=True)
    for iter1 in range(100):
#         print(iter1)
        #E step
        M=(-0.5*(1/sigma)@Temp1.transpose()+(mu/sigma)@train_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi*sigma)-0.5*(np.sum(mu*mu/sigma,axis=1,keepdims=True))).transpose()
        Max_M=np.max(M, axis=1,keepdims=True)
        F=M-Max_M
        F=np.exp(F)
        L=np.sum(F,axis=1,keepdims=True)
        F=F/L
        F+=0.00001*np.ones(F.shape)# add a small noise avoid 0 denorminator
        F=F/np.sum(F,axis=1,keepdims=True)
        #Log likelyhood
        L=np.log(L)
        L=np.sum(L)+np.sum(Max_M)
        if abs(L_old-L)/abs(L)<0.0001:
            break
        L_old=L
        #M_step
        Sum_F = np.sum(F,axis=0,keepdims=True)
        Pi=Sum_F/N
        mu=F.transpose()@train_X_vec_selected
        mu=mu/Sum_F.transpose()
        Temp2 =-2*np.sum(F*(train_X_vec_selected@mu.transpose()),axis=0,keepdims=True)
        Temp3=Sum_F*(np.sum(mu*mu,axis=1,keepdims=True)).transpose()
        sigma = F.transpose()@Temp1+Temp2.transpose()+Temp3.transpose()
        sigma = sigma/Sum_F.transpose()/d
    ##Figure our how the groups of GMM correspond to true lable
    train_cluster_result=np.argmax(F,axis=1)
    G_to_L=[]
    for k1 in range(K):
        temp=train_cluster_result[ind_list1[k1]]
        current=0
        relationship=[k1,k1]
        for k2 in range(K):
            if np.sum(temp==k2)>current:
                current=np.sum(temp==k2)
                relationship[1]=k2
        G_to_L.append(relationship)    
    #testing
    N_test=test_X_vec_selected.shape[0]
    test_Temp1=np.sum(test_X_vec_selected*test_X_vec_selected,axis=1,keepdims=True)
    test_M=(-0.5*(1/sigma)@test_Temp1.transpose()+(mu/sigma)@test_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi*sigma)-0.5*(np.sum(mu*mu/sigma,axis=1,keepdims=True))).transpose()
    test_Max_M=np.max(test_M, axis=1,keepdims=True)
    test_F=test_M-test_Max_M
    test_F=np.exp(test_F)
    test_L=np.sum(test_F,axis=1,keepdims=True)
    test_F=test_F/test_L
    test_L=np.log(test_L)
    test_L=np.sum(test_L)+np.sum(test_Max_M)
    test_result=np.argmax(test_F,axis=1)
    trans_test_y_selected=np.zeros([N_test])
    for i in range(N_test):
        trans_test_y_selected[i]=G_to_L[test_y_selected[i]][1]
    error_rate_list1.append(1-np.sum(test_result==trans_test_y_selected)/N_test)
print("The mean error rate in 500 experiments is ",np.mean(error_rate_list1))
print("The standard deviation of the error rate in 500 experiments is ",np.std(error_rate_list1))

The mean error rate in 500 experiments is  0.1446904066939093
The standard deviation of the error rate in 500 experiments is  0.05292906532868437


### Mixture of diagnal Gaussians

In [46]:
##initialization
K=5
N=train_X_vec_selected.shape[0]
d=196
mu=np.random.rand(K,d)
sigma=np.random.rand(K,196)
Pi=np.ones(K)/5
ind_list2=[]
for i in range(5):
    ind_list2.append(np.where(train_y_selected==i)[0])
##EM algorithm
L_old=1
Temp1=train_X_vec_selected*train_X_vec_selected
for iter1 in range(100):
    print(iter1)
    #E step
    M=(-0.5*(1/sigma)@Temp1.transpose()+(mu/sigma)@train_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi)-0.5*(np.sum(mu*mu/sigma+np.log(sigma),axis=1,keepdims=True))).transpose()
    Max_M=np.max(M, axis=1,keepdims=True)
    F=M-Max_M
    F=np.exp(F)
    L=np.sum(F,axis=1,keepdims=True)
    F=F/L
    F+=0.00001*np.ones(F.shape)# add a small noise avoid 0 denorminator
    F=F/np.sum(F,axis=1,keepdims=True)
    #Log likelyhood
    L=np.log(L)
    L=np.sum(L)+np.sum(Max_M)
    if abs(L_old-L)/abs(L)<0.0001:
        break
    L_old=L
    #M_step
    SUM_F = np.sum(F,axis=0,keepdims=True)
    Pi=SUM_F/N
    mu=F.transpose()@train_X_vec_selected
    mu=mu/SUM_F.transpose()
    Temp2=-2*mu*(F.transpose()@train_X_vec_selected)
    Temp3=mu*mu*(F.transpose()@Temp1)
    sigma = F.transpose()@Temp1+Temp2+Temp3
    sigma = sigma/SUM_F.transpose()/d+0.05
##Figure our how the groups of GMM correspond to true lable
train_cluster_result=np.argmax(F,axis=1)
G_to_L=[]
for k1 in range(K):
    temp=train_cluster_result[ind_list2[k1]]
    current=0
    relationship=[k1,k1]
    for k2 in range(K):
        if np.sum(temp==k2)>current:
            current=np.sum(temp==k2)
            relationship[1]=k2
    G_to_L.append(relationship)    

#testing
N_test=test_X_vec_selected.shape[0]
test_Temp1=test_X_vec_selected*test_X_vec_selected

test_M=(-0.5*(1/sigma)@test_Temp1.transpose()+(mu/sigma)@test_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi)-0.5*(np.sum(mu*mu/sigma+np.log(sigma),axis=1,keepdims=True))).transpose()
test_Max_M=np.max(test_M, axis=1,keepdims=True)
test_F=test_M-test_Max_M
test_F=np.exp(test_F)
test_L=np.sum(test_F,axis=1,keepdims=True)
test_F=test_F/test_L
test_L=np.log(test_L)
test_L=np.sum(test_L)+np.sum(test_Max_M)
test_result=np.argmax(test_F,axis=1)

trans_test_y_selected=np.zeros([N_test])
for i in range(N_test):
    trans_test_y_selected[i]=G_to_L[test_y_selected[i]][1]
print("The testing error is ", 1-np.sum(test_result==trans_test_y_selected)/N_test)
print("The Log Likelyhood is ",test_L)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
The testing error is  0.11733800350262702
The Log Likelyhood is  203869.15005684394


In [48]:
##500 experiments
error_rate_list2=[]
for xx in range(500):
#     print(xx)
    ##initialization
    K=5
    N=train_X_vec_selected.shape[0]
    d=196
    mu=np.random.rand(K,d)
    sigma=np.random.rand(K,196)
    Pi=np.ones(K)/5
    ind_list2=[]
    for i in range(5):
        ind_list2.append(np.where(train_y_selected==i)[0])
    ##EM algorithm
    L_old=1
    Temp1=train_X_vec_selected*train_X_vec_selected
    for iter1 in range(100):
#         print(iter1)
        #E step

        M=(-0.5*(1/sigma)@Temp1.transpose()+(mu/sigma)@train_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi)-0.5*(np.sum(mu*mu/sigma+np.log(sigma),axis=1,keepdims=True))).transpose()
        Max_M=np.max(M, axis=1,keepdims=True)
        F=M-Max_M
        F=np.exp(F)
        L=np.sum(F,axis=1,keepdims=True)
        F=F/L
        F+=0.00001*np.ones(F.shape)# add a small noise avoid 0 denorminator
        F=F/np.sum(F,axis=1,keepdims=True)
        #Log likelyhood
        L=np.log(L)
        L=np.sum(L)+np.sum(Max_M)
        if abs(L_old-L)/abs(L)<0.0001:
            break
        L_old=L
        #M_step
        SUM_F = np.sum(F,axis=0,keepdims=True)
        Pi=SUM_F/N
        mu=F.transpose()@train_X_vec_selected
        mu=mu/SUM_F.transpose()
        Temp2=-2*mu*(F.transpose()@train_X_vec_selected)
        Temp3=mu*mu*(F.transpose()@Temp1)
        sigma = F.transpose()@Temp1+Temp2+Temp3
        sigma = sigma/SUM_F.transpose()/d+0.05
    ##Figure our how the groups of GMM correspond to true lable
    train_cluster_result=np.argmax(F,axis=1)
    G_to_L=[]
    for k1 in range(K):
        temp=train_cluster_result[ind_list2[k1]]
        current=0
        relationship=[k1,k1]
        for k2 in range(K):
            if np.sum(temp==k2)>current:
                current=np.sum(temp==k2)
                relationship[1]=k2
        G_to_L.append(relationship)    

    #testing
    N_test=test_X_vec_selected.shape[0]
    test_Temp1=test_X_vec_selected*test_X_vec_selected

    test_M=(-0.5*(1/sigma)@test_Temp1.transpose()+(mu/sigma)@test_X_vec_selected.transpose()-0.5*d*np.log(2*math.pi)-0.5*(np.sum(mu*mu/sigma+np.log(sigma),axis=1,keepdims=True))).transpose()
    test_Max_M=np.max(test_M, axis=1,keepdims=True)
    test_F=test_M-test_Max_M
    test_F=np.exp(test_F)
    test_L=np.sum(test_F,axis=1,keepdims=True)
    test_F=test_F/test_L
    test_L=np.log(test_L)
    test_L=np.sum(test_L)+np.sum(test_Max_M)
    test_result=np.argmax(test_F,axis=1)

    trans_test_y_selected=np.zeros([N_test])
    for i in range(N_test):
        trans_test_y_selected[i]=G_to_L[test_y_selected[i]][1]
    error_rate_list2.append(1-np.sum(test_result==trans_test_y_selected)/N_test)
print("The mean error rate in 500 experiments is ",np.mean(error_rate_list2))
print("The standard deviation of the error rate in 500 experiments is ",np.std(error_rate_list2))

The mean error rate in 500 experiments is  0.1332387624051372
The standard deviation of the error rate in 500 experiments is  0.04137207095309059
